# Core 

> Minimal pipeline for Diffusion Guidance experiments.

In [ ]:
#| default_exp guidance

In [ ]:
#| export
from dynamic_cfg.kdiff import *
from dynamic_cfg.schedules import name2schedule
from dynamic_cfg.normalizers import name2norm

In [ ]:
#| export

class DynamicCFG:
    def __init__(self, norm_name, schedule_name):
        self.normalzier = name2norm[norm_name]()
        self.schedules = name2schedule[schedule_name]()

    def guide(self, cond, uncond, ts):
        """Applies dynamic Classifier-free Guidance.
        """
        # set the conditional and unconditional vectors
        self.normalzier.set_latents(u=uncond, t=cond)
        # compute the update: (cond - uncond)
        self.normalzier.compute_update()

        # apply optional pre-processing
        if self.normalzier.has_preproc:
            self.normalzier.pre_proc()

        # get the current, scheduled guidance value 
        guide_scale = self.scheduler.value_at(ts)
        # run classifier-free guidance 
        pred = self.normalzier.u + (guide_scale * self.normalzier.diff)
        self.normalzier.set_pred(pred)

        # apply optional post-processing
        if self.normalzier.has_postproc:
            self.normalzier.post_proc()

        return self.normalzier.get_pred()

    def update_sched_kwargs(self, other_kwargs):
        self.schedules.update_sched_kwargs(other_kwargs)
    def set_timesteps(self, num_steps):
        self.schedules.set_num_steps(num_steps)
        self.schedules.set_guidance_schedule()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()